<a href="https://colab.research.google.com/github/Evaabee/data-feature/blob/main/WeeklyAssignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from google.colab import userdata

#api keys
aviationstack_api_key = userdata.get("AVIATIONSTACK_API_KEY")
visualcrossing_api_key = userdata.get("VISUALCROSSING_API_KEY")

if aviationstack_api_key:
    print("The AviationStack API key has been retrieved successfully!")
else:
    print("Failed to retrieve the AviationStack API key.")

if visualcrossing_api_key:
    print("The VisualStack API key has been retrieved successfully!")
else:
    print("Failed to retrieve the VisualStack API key.")

The AviationStack API key has been retrieved successfully!
The VisualStack API key has been retrieved successfully!


In [3]:
#function to check if weather conditions might cause delays
def is_bad_weather(precip_prob, wind_speed, conditions):
    # Define thresholds for bad weather
    if precip_prob > 50 or wind_speed > 20 or 'rain' in conditions.lower() or 'snow' in conditions.lower():
        return True
    return False

In [4]:
#fetch weather data using Visual Crossing api
def fetch_weather(city_name):
    weather_url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city_name}/today?key={visualcrossing_api_key}"
    weather_response = requests.get(weather_url)
    return weather_response.json()

In [6]:
#get user input for the flight number
flight_number = input("Enter the flight number (e.g., EK5): ")

Enter the flight number (e.g., EK5): RC470


In [7]:
#fetch flight status from AviationStack api
flight_url = f"http://api.aviationstack.com/v1/flights?access_key={aviationstack_api_key}&flight_iata={flight_number}"
flight_response = requests.get(flight_url)
flight_data = flight_response.json()

if flight_data['data']:
    flight_info = flight_data['data'][0]

    flight_status = flight_info['flight_status']


    departure_city = flight_info['departure']['airport']
    arrival_city = flight_info['arrival']['airport']

    print(f"\nFlight Status for {flight_number}: {flight_status}")
    print(f"Departure City: {departure_city}")
    print(f"Arrival City: {arrival_city}")

    #Fetch and display weather for departure city
    departure_weather = fetch_weather(departure_city)
    if departure_weather:
        dep_conditions = departure_weather['days'][0]['conditions']
        dep_precip_prob = departure_weather['days'][0]['precipprob']
        dep_wind_speed = departure_weather['days'][0]['windspeed']

        print(f"\nWeather in {departure_city}: {dep_conditions}")
        print(f"Precipitation Probability: {dep_precip_prob}%")
        print(f"Wind Speed: {dep_wind_speed} mph")

        if is_bad_weather(dep_precip_prob, dep_wind_speed, dep_conditions):
            print(f"Bad weather detected in {departure_city}.")

  #fetch and display weather for arrival city
    arrival_weather = fetch_weather(arrival_city)
    if arrival_weather:
        arr_conditions = arrival_weather['days'][0]['conditions']
        arr_precip_prob = arrival_weather['days'][0]['precipprob']
        arr_wind_speed = arrival_weather['days'][0]['windspeed']

        print(f"\nWeather in {arrival_city}: {arr_conditions}")
        print(f"Precipitation Probability: {arr_precip_prob}%")
        print(f"Wind Speed: {arr_wind_speed} mph")

        if is_bad_weather(arr_precip_prob, arr_wind_speed, arr_conditions):
            print(f"Bad weather detected in {arrival_city}.")
            if flight_status == 'delayed':
                print("The flight is already delayed, and weather at the arrival city might be a factor.")
            else:
                print("The flight is currently on time, but bad weather at the arrival city might cause a delay.")
else:
    print(f"No flight data available for flight {flight_number}.")


Flight Status for RC470: scheduled
Departure City: Vagar
Arrival City: Billund

Weather in Vagar: Rain, Partially cloudy
Precipitation Probability: 100.0%
Wind Speed: 6.3 mph
Bad weather detected in Vagar.

Weather in Billund: Partially cloudy
Precipitation Probability: 0.0%
Wind Speed: 20.8 mph
Bad weather detected in Billund.
The flight is currently on time, but bad weather at the arrival city might cause a delay.
